### Gathering MRT data

In [1]:
import pandas as pd
pd.set_option("max_columns", None)
import numpy as np
import requests
import time

In [2]:
df_mrt = pd.read_excel("MRT-Stations.xlsx")
df_mrt.head()

,Code,Station Name
0,NS1 EW24,Jurong East
1,NS2,Bukit Batok
2,NS3,Bukit Gombak
3,NS4 BP1,Choa Chu Kang
4,NS5,Yew Tee


In [3]:
mrt_address_list = df_mrt["Station Name"] + " MRT"
mrt_address_list.head()

0      Jurong East MRT
1      Bukit Batok MRT
2     Bukit Gombak MRT
3    Choa Chu Kang MRT
4          Yew Tee MRT
Name: Station Name, dtype: object

In [4]:
def get_coordinate(address):
    response = requests.get(f"https://developers.onemap.sg/commonapi/search?searchVal={address}&returnGeom=Y&getAddrDetails=Y&pageNum=1").json()
    return response["results"][0]["POSTAL"], response["results"][0]["LATITUDE"], response["results"][0]["LONGITUDE"]

In [5]:
count = 0
mrt_coordinates_list = []
mrt_error_list = []

for address in mrt_address_list:
    try:
        coords = get_coordinate(address)
        mrt_coordinates_list.append(coords)
        count += 1
        time.sleep(0.3)
        print(f"{count} of {len(mrt_address_list)} datapoints obtained. {len(mrt_error_list)} errors encountered.")
                        
    except:
        mrt_error_list.append(address)
        mrt_coordinates_list.append(None)
        print(f"{len(mrt_error_list)} errors encountered.")
        
print(f"Completed. There were {len(mrt_error_list)} errors.")

1 of 122 datapoints obtained. 0 errors encountered.
2 of 122 datapoints obtained. 0 errors encountered.
3 of 122 datapoints obtained. 0 errors encountered.
4 of 122 datapoints obtained. 0 errors encountered.
5 of 122 datapoints obtained. 0 errors encountered.
6 of 122 datapoints obtained. 0 errors encountered.
7 of 122 datapoints obtained. 0 errors encountered.
8 of 122 datapoints obtained. 0 errors encountered.
9 of 122 datapoints obtained. 0 errors encountered.
10 of 122 datapoints obtained. 0 errors encountered.
11 of 122 datapoints obtained. 0 errors encountered.
12 of 122 datapoints obtained. 0 errors encountered.
13 of 122 datapoints obtained. 0 errors encountered.
14 of 122 datapoints obtained. 0 errors encountered.
15 of 122 datapoints obtained. 0 errors encountered.
16 of 122 datapoints obtained. 0 errors encountered.
17 of 122 datapoints obtained. 0 errors encountered.
18 of 122 datapoints obtained. 0 errors encountered.
19 of 122 datapoints obtained. 0 errors encountered.
20

In [6]:
df_mrt_coordinates = pd.DataFrame(mrt_coordinates_list, columns = ["Postal", "Latitude", "Longitude"])
df_mrt_coordinates.head()

,Postal,Latitude,Longitude
0,609690,1.33315281585758,103.742286332403
1,659958,1.34903331201636,103.749566478309
2,659083,1.35861159094192,103.751790910733
3,689810,1.38536316540225,103.744370779756
4,689715,1.39753506936297,103.747405150236


In [7]:
#Join obtained coords with df_mrt
df_mrt = df_mrt.join(df_mrt_coordinates)
df_mrt = df_mrt.drop(columns = ["Code"])
df_mrt.head()

,Station Name,Postal,Latitude,Longitude
0,Jurong East,609690,1.33315281585758,103.742286332403
1,Bukit Batok,659958,1.34903331201636,103.749566478309
2,Bukit Gombak,659083,1.35861159094192,103.751790910733
3,Choa Chu Kang,689810,1.38536316540225,103.744370779756
4,Yew Tee,689715,1.39753506936297,103.747405150236


In [8]:
#df_mrt.to_csv("MRT-with-coordinates.csv", index=False)
df_mrt = pd.read_csv("MRT-with-coordinates.csv")

### Obtaining Nearest MRT Station and Distance to Station

In [9]:
df_hdb = pd.read_csv("Resale-list-with-coordinates-and-height.csv")
df_hdb.head()

,month,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,Postal,Latitude,Longitude,max_floor_lvl,height_index
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,44.0,Improved,1979,61 years 04 months,232000.0,560406,1.362005,103.85388,12,4.0
1,2017-05,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,44.0,Improved,1979,61 years 01 month,235000.0,560406,1.362005,103.85388,12,4.0
2,2018-03,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,44.0,Improved,1979,60 years 02 months,202000.0,560406,1.362005,103.85388,12,1.0
3,2018-03,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,44.0,Improved,1979,60 years 02 months,210000.0,560406,1.362005,103.85388,12,1.0
4,2018-05,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,44.0,Improved,1979,60 years 01 month,220000.0,560406,1.362005,103.85388,12,3.0


In [10]:
df_hdb.shape[0]

134393

In [11]:
df_hdb.dtypes

month                   object
town                    object
flat_type               object
block                   object
street_name             object
floor_area_sqm         float64
flat_model              object
lease_commence_date      int64
remaining_lease         object
resale_price           float64
Postal                   int64
Latitude               float64
Longitude              float64
max_floor_lvl            int64
height_index           float64
dtype: object

In [12]:
df_mrt.dtypes

Station Name     object
Postal            int64
Latitude        float64
Longitude       float64
dtype: object

In [13]:
df_hdb.Postal = pd.to_numeric(df_hdb.Postal, errors = "coerce")
df_hdb.Latitude = df_hdb.Latitude.astype(float)
df_hdb.Longitude = df_hdb.Longitude.astype(float)
df_mrt.Latitude = df_mrt.Latitude.astype(float)
df_mrt.Longitude = df_mrt.Longitude.astype(float)
print(df_hdb.dtypes)
print(df_mrt.dtypes)

month                   object
town                    object
flat_type               object
block                   object
street_name             object
floor_area_sqm         float64
flat_model              object
lease_commence_date      int64
remaining_lease         object
resale_price           float64
Postal                   int64
Latitude               float64
Longitude              float64
max_floor_lvl            int64
height_index           float64
dtype: object
Station Name     object
Postal            int64
Latitude        float64
Longitude       float64
dtype: object


In [14]:
#Using sklearn's haversine distance function to obtain nearest MRT station and distance to the station

from sklearn.metrics.pairwise import haversine_distances

nearest_station = []
shortest_distance = []
for lat, long in zip(df_hdb["Latitude"], df_hdb["Longitude"]):
    latlong = [[lat,long]]
    result = haversine_distances(np.radians(latlong), np.radians(df_mrt[["Latitude", "Longitude"]])) * 6371000/1000
    distance = np.amin(result)
    station = np.where(result == distance)[1][0]
    shortest_distance.append(round(distance,2))
    nearest_station.append(station)
df_hdb['Nearest_Station'] = nearest_station
df_hdb['Distance_to_MRT'] = shortest_distance

In [15]:
df_hdb.tail()

,month,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,Postal,Latitude,Longitude,max_floor_lvl,height_index,Nearest_Station,Distance_to_MRT
134388,2022-08,WOODLANDS,4 ROOM,693A,WOODLANDS AVE 6,92.0,Model A,2018,95 years 03 months,540000.0,731693,1.438782,103.803708,13,3.0,8,0.36
134389,2022-08,WOODLANDS,4 ROOM,691A,WOODLANDS DR 73,93.0,Model A,2018,94 years 11 months,555000.0,731691,1.440304,103.806660,14,3.0,8,0.63
134390,2022-08,WOODLANDS,4 ROOM,784B,WOODLANDS RISE,93.0,Model A,2018,95 years,538000.0,732784,1.445521,103.804854,14,5.0,8,0.70
134391,2022-08,WOODLANDS,4 ROOM,784B,WOODLANDS RISE,93.0,Model A,2018,95 years,535000.0,732784,1.445521,103.804854,14,2.0,8,0.70
134392,2022-08,WOODLANDS,5 ROOM,783A,WOODLANDS RISE,113.0,Improved,2018,95 years 01 month,605888.0,731783,1.447417,103.803742,16,6.0,8,0.82


In [16]:
df_mrt.head()

,Station Name,Postal,Latitude,Longitude
0,Jurong East,609690,1.333153,103.742286
1,Bukit Batok,659958,1.349033,103.749566
2,Bukit Gombak,659083,1.358612,103.751791
3,Choa Chu Kang,689810,1.385363,103.744371
4,Yew Tee,689715,1.397535,103.747405


In [17]:
#Dropping MRT coordinates as there is no further need for them
df_mrt = df_mrt.drop(columns = ["Postal", "Latitude", "Longitude"])
#Create a new column mirroring the index to join both dataframes
df_mrt["IndexCopy"] = df_mrt.index
df_mrt.head()

,Station Name,IndexCopy
0,Jurong East,0
1,Bukit Batok,1
2,Bukit Gombak,2
3,Choa Chu Kang,3
4,Yew Tee,4


In [18]:
#Join both dataframes
df_final = df_hdb.merge(df_mrt, left_on= df_hdb["Nearest_Station"], right_on = df_mrt["IndexCopy"])
df_final.head()

,key_0,month,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,Postal,Latitude,Longitude,max_floor_lvl,height_index,Nearest_Station,Distance_to_MRT,Station Name,IndexCopy
0,14,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,44.0,Improved,1979,61 years 04 months,232000.0,560406,1.362005,103.85388,12,4.0,14,1.0,Ang Mo Kio,14
1,14,2017-05,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,44.0,Improved,1979,61 years 01 month,235000.0,560406,1.362005,103.85388,12,4.0,14,1.0,Ang Mo Kio,14
2,14,2018-03,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,44.0,Improved,1979,60 years 02 months,202000.0,560406,1.362005,103.85388,12,1.0,14,1.0,Ang Mo Kio,14
3,14,2018-03,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,44.0,Improved,1979,60 years 02 months,210000.0,560406,1.362005,103.85388,12,1.0,14,1.0,Ang Mo Kio,14
4,14,2018-05,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,44.0,Improved,1979,60 years 01 month,220000.0,560406,1.362005,103.85388,12,3.0,14,1.0,Ang Mo Kio,14


In [19]:
df_final = df_final.drop(columns = ["key_0", "Nearest_Station", "IndexCopy"])
df_final.tail()

,month,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,Postal,Latitude,Longitude,max_floor_lvl,height_index,Distance_to_MRT,Station Name
134388,2021-10,BUKIT TIMAH,4 ROOM,3,QUEEN'S RD,91.0,Improved,1974,51 years 07 months,550000.0,266734,1.322878,103.811648,12,4.0,0.42,Botanic Gardens
134389,2021-10,BUKIT TIMAH,4 ROOM,3,QUEEN'S RD,91.0,Improved,1974,51 years 06 months,483000.0,266734,1.322878,103.811648,12,2.0,0.42,Botanic Gardens
134390,2022-03,BUKIT TIMAH,4 ROOM,3,QUEEN'S RD,91.0,Improved,1974,51 years 02 months,580000.0,266734,1.322878,103.811648,12,2.0,0.42,Botanic Gardens
134391,2022-06,BUKIT TIMAH,4 ROOM,3,QUEEN'S RD,91.0,Improved,1974,50 years 10 months,528000.0,266734,1.322878,103.811648,12,2.0,0.42,Botanic Gardens
134392,2022-07,BUKIT TIMAH,4 ROOM,3,QUEEN'S RD,91.0,Improved,1974,50 years 10 months,560000.0,266734,1.322878,103.811648,12,3.0,0.42,Botanic Gardens


In [20]:
df_final.shape[0]

134393

In [21]:
#df_final.to_csv("Resale_Data_Final_Cleaned.csv", index = False)
df_final = pd.read_csv("Resale_Data_Final_Cleaned.csv")

---

### Obtaining MRT Travel Duration

In [22]:
import requests
import time
import googlemaps

In [23]:
gmaps = googlemaps.Client(key="your-key")

In [24]:
df_mrt.head()

,Station Name,IndexCopy
0,Jurong East,0
1,Bukit Batok,1
2,Bukit Gombak,2
3,Choa Chu Kang,3
4,Yew Tee,4


In [31]:
duration_list = []
error_list = []
for station in df_mrt["Station Name"]:
    try:
        origin = station + " MRT"
        response = gmaps.distance_matrix(origins = origin, destinations = "City Hall MRT", mode = "transit", transit_mode = "rail", departure_time = "1667206800")
        #departure_time = 1667260800 is Unix timestamp and is required by the api to be set as current time or sometime in the future
        #1667260800 equates to Tue Nov 01 2022 08:00:00 GMT+0800 (Singapore Standard Time)
        duration = response["rows"][0]["elements"][0]["duration"]["text"]
        duration_list.append(duration)
        time.sleep(0.2)
    except:
        print("Error at", origin)
        error_list.append(origin)
        duration_list.append(None)
df_mrt["Duration"] = duration_list
df_mrt.head()

Error at City Hall MRT
Error at HarbourFront MRT
Error at Pasir Panjang MRT
Error at Downtown MRT


,Station Name,IndexCopy,Duration
0,Jurong East,0,29 mins
1,Bukit Batok,1,40 mins
2,Bukit Gombak,2,41 mins
3,Choa Chu Kang,3,46 mins
4,Yew Tee,4,45 mins


In [32]:
error_list

['City Hall MRT', 'HarbourFront MRT', 'Pasir Panjang MRT', 'Downtown MRT']

In [ ]:
#Even when repeated, these stations do not receive a response from the API.
#Besides City Hall station (for which destination is same as origin), the reason for null response is not known.
#Using Google directions search, we obtain:
#Pasir Panjang = 34
#HarbourFront = 25
#Downtown = 13

In [35]:
pd.set_option('display.max_rows', 130)
df_mrt_duration = df_mrt.copy()
df_mrt_duration["Duration"] = duration_list
df_mrt_duration

,Station Name,IndexCopy,Duration
0,Jurong East,0,29 mins
1,Bukit Batok,1,40 mins
2,Bukit Gombak,2,41 mins
3,Choa Chu Kang,3,46 mins
4,Yew Tee,4,45 mins
5,Kranji,5,51 mins
6,Marsiling,6,48 mins
7,Woodlands,7,46 mins
8,Admiralty,8,42 mins
9,Sembawang,9,41 mins


In [36]:
df_mrt_duration.loc[(df_mrt_duration["Station Name"] == "City Hall"), "Duration"] = "0 mins"
df_mrt_duration.loc[(df_mrt_duration["Station Name"] == "HarbourFront"), "Duration"] = "22 mins"
df_mrt_duration.loc[(df_mrt_duration["Station Name"] == "Pasir Panjang"), "Duration"] = "36 mins"
df_mrt_duration.loc[(df_mrt_duration["Station Name"] == "Downtown"), "Duration"] = "15 mins"
df_mrt_duration.loc[(df_mrt_duration["Station Name"] == "Woodlands South"), "Duration"] = "60 mins"

In [38]:
df_mrt_duration = df_mrt_duration.drop(columns = ["IndexCopy"])

In [39]:
#df_mrt_duration.to_csv("MRT_Duration.csv", index = False)
df_mrt_duration = pd.read_csv("MRT_Duration.csv")

---

### Merging Main DataFrame with Travel Duration

In [40]:
df_marged_with_duration = df_final.merge(df_mrt_duration, left_on= df_final["Station Name"], right_on= df_mrt_duration["Station Name"])
df_marged_with_duration.head()

,key_0,month,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,Postal,Latitude,Longitude,max_floor_lvl,height_index,Distance_to_MRT,Station Name_x,Station Name_y,Duration
0,Ang Mo Kio,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,44.0,Improved,1979,61 years 04 months,232000.0,560406,1.362005,103.85388,12,4.0,1.0,Ang Mo Kio,Ang Mo Kio,26 mins
1,Ang Mo Kio,2017-05,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,44.0,Improved,1979,61 years 01 month,235000.0,560406,1.362005,103.85388,12,4.0,1.0,Ang Mo Kio,Ang Mo Kio,26 mins
2,Ang Mo Kio,2018-03,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,44.0,Improved,1979,60 years 02 months,202000.0,560406,1.362005,103.85388,12,1.0,1.0,Ang Mo Kio,Ang Mo Kio,26 mins
3,Ang Mo Kio,2018-03,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,44.0,Improved,1979,60 years 02 months,210000.0,560406,1.362005,103.85388,12,1.0,1.0,Ang Mo Kio,Ang Mo Kio,26 mins
4,Ang Mo Kio,2018-05,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,44.0,Improved,1979,60 years 01 month,220000.0,560406,1.362005,103.85388,12,3.0,1.0,Ang Mo Kio,Ang Mo Kio,26 mins


In [41]:
df_marged_with_duration.shape[0]

134393

In [42]:
df_marged_with_duration = df_marged_with_duration.drop(columns = ["key_0", "Station Name_y"])
df_marged_with_duration = df_marged_with_duration.rename(columns = {"month": "resale_date", "Station Name_x": "station_name", "Duration": "travel_time" })
df_marged_with_duration.head()

,resale_date,town,flat_type,block,street_name,floor_area_sqm,flat_model,lease_commence_date,remaining_lease,resale_price,Postal,Latitude,Longitude,max_floor_lvl,height_index,Distance_to_MRT,station_name,travel_time
0,2017-01,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,44.0,Improved,1979,61 years 04 months,232000.0,560406,1.362005,103.85388,12,4.0,1.0,Ang Mo Kio,26 mins
1,2017-05,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,44.0,Improved,1979,61 years 01 month,235000.0,560406,1.362005,103.85388,12,4.0,1.0,Ang Mo Kio,26 mins
2,2018-03,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,44.0,Improved,1979,60 years 02 months,202000.0,560406,1.362005,103.85388,12,1.0,1.0,Ang Mo Kio,26 mins
3,2018-03,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,44.0,Improved,1979,60 years 02 months,210000.0,560406,1.362005,103.85388,12,1.0,1.0,Ang Mo Kio,26 mins
4,2018-05,ANG MO KIO,2 ROOM,406,ANG MO KIO AVE 10,44.0,Improved,1979,60 years 01 month,220000.0,560406,1.362005,103.85388,12,3.0,1.0,Ang Mo Kio,26 mins


In [43]:
df_marged_with_duration.to_csv("Resale_Data_with_travel_duration.csv", index = False)